In [45]:
import json

# Replace 'your_file.json' with your actual JSON file name
file_name = "vectorized_summary.json"

# Open the JSON file and load the data
with open(file_name, "r") as file:
    data = json.load(file)

# Now 'data' is a Python dictionary containing the contents of the JSON file
# print(data)

In [46]:
# List of your dictionaries
data_list = data

# Extract vectors from the dictionaries
vectors = [item["vector"] for item in data_list]

In [47]:
import numpy as np
from scipy.cluster.vq import kmeans, vq

# ANYA
from usearch.index import Index

# Your list of vectors
your_vectors = vectors

# 1. Create an Index and Cluster Vectors
index = Index(ndim=len(your_vectors[0]))  # Assuming all vectors have the same dimension

for key, vector in enumerate(your_vectors):
    index.add(key, np.array(vector))

clustering = index.cluster(max_count=100)  # Perform clustering
clustering.matches

usearch.BatchMatches(689 across 689 queries)

In [48]:

# Assuming 'data' is your matrix of vectors. Replace this with your actual data.
# Example: data = np.array([[1, 2], [2, 3], [1, 4], [5, 6], [7, 8], [9, 10]])


# data = np.array(vectors * 10)
data = np.vstack(index.get(np.array(index.keys), dtype=np.float32))

# Number of clusters
num_clusters = 15

# Compute K-Means with 'num_clusters' clusters
centroids, _ = kmeans(data, num_clusters)

# Assign each sample to a cluster
idx, _ = vq(data, centroids)

# 'idx' is an array of cluster indices corresponding to each vector in your matrix
print("Cluster indices for each vector:", idx)


### -----
# def is_vector_in_cluster(vector, centroids, threshold):
#     # Normalize or preprocess the vector if necessary
#     # vector = preprocess(vector)  # Uncomment and define preprocess if needed

#     # Find the nearest centroid and the distance to it
#     centroid_idx, distance = vq(np.array([vector]), centroids)

#     # Check if the distance is within the threshold
#     is_within_cluster = distance[0] <= threshold
#     return is_within_cluster, centroid_idx[0]


# # Example usage:
# new_vector = np.array(vectors[0])  # Replace with your new vector
# threshold = 150  # Define your threshold

# is_in_cluster, cluster_idx = is_vector_in_cluster(new_vector, centroids, threshold)
# print("Is in cluster:", is_in_cluster, "Cluster index:", cluster_idx)

Cluster indices for each vector: [ 5  0  5  9  0  9  2  5  9  5  2  9  0  5  5  5  0  0  2  9  9  9  2  2
  0  2  2  5  6  2  6  6  1  2  0  2  9  5  2  5  6  9  2  0  5  0  9  0
  2  5  6  0  6  6  6  0  2  6  0  9  0  9  0  5  9  5  5  5  0  0  2 12
  0  5  9  9  5 10  2  9  2  2  5  5  0  5  5  8  5  5  5  0  9  5  5  0
  9  5  2  5  5  5  5  5  5  5  5  5  2  5  5  5 13  0  2  1  9  9  1  5
  2  5  5  0  9 12  5  5  9  2  0  2  9  5  9  0 13  5  5  0  9  9  9  9
  5  0  7  0  9  0  0  0  9  0  9  2  2  2  5  2  5  2  2  7  2  2  5  0
  5  5  9  9  5  0  5  5  0  9  5  5  7  0  0  9  9  5  5  9  2  7  2  5
  5  5  0  0  0  9  9 13  5  2  2  9  9  9 12 13  0  0  9  5  0  9  9  9
  0  2  5 12  5  9  2  2  2  2  2  5  9 14  2  9  9  2  0  2  2  2  2  2
  5  2  9  2  2  5  2  2  2 14  9  2 12  9  0 13 13 12  0  9  2  8  0 13
  2  0  0 14  5  2  0  0  9  2  9  2  9  2  2 13  0  2  9 13 13  5  9 13
 13  9  5  9  5  1  9  5  0  2  0 13  0  0  9  1  0  2 11 13  1  5  3  0
  3  0  3 11 11  5

In [49]:
import random

# Assuming 'idx' is the array of cluster indices you obtained from clustering
clustered_data = [[] for _ in range(num_clusters)]

# Group data items by their cluster, removing the 'vector' field
for i, item in enumerate(data_list):
    cluster_index = idx[i]
    item_without_vector = item.copy()  # Copy to avoid modifying the original data
    item_without_vector.pop("vector", None)  # Remove the 'vector' field
    clustered_data[cluster_index].append(item_without_vector)

# Select one random item from each cluster and compile links
random_items = []
cluster_links = []

for cluster in clustered_data:
    random_item = random.choice(
        cluster
    ).copy()  # Copy to avoid modifying the original data
    random_item.pop("vector", None)  # Remove the 'vector' field
    random_items.append(random_item)

    links = [item["link"] for item in cluster]
    cluster_links.append(links)

# random_items now contains one random item from each cluster
# cluster_links is a list of lists, each containing the links of all items in each cluster

In [51]:
# Prepare the final list of cluster results
cluster_results = []

for i, cluster in enumerate(clustered_data):
    if cluster:  # Check if the cluster is not empty
        # Select a random item for title, description, and date
        random_item = random.choice(cluster)

        # Create the cluster dictionary
        cluster_dict = {
            "title": random_item["title"],
            "description": random_item["summary"],  # Rename 'excerpt' to 'description'
            "date": random_item["published_date"],  # Rename 'published_date' to 'date'
            "sources": cluster_links[i],  # Add all links in the cluster
        }

        # Add the dictionary to the results list
        cluster_results.append(cluster_dict)

# 'cluster_results' now contains a list of dictionaries, each representing a cluster with the desired fields

In [52]:
import json

# Prepare the final list of cluster results (assuming this is already done)
# cluster_results = [...]

# Specify the filename for the JSON file
filename = f"cluster_results_{num_clusters}.json"

# Write the cluster results to a JSON file
with open(filename, "w") as file:
    json.dump(cluster_results, file, indent=4)

print(f"Cluster results saved to {filename}")

Cluster results saved to cluster_results_15.json
